In [57]:
import pandas as pd 

Task 1: Load and inspect the dataset

In [58]:
scores = pd.read_csv('data_safe_copy.csv')
scores

,student_id,cohort,module,assignment,score
0,S001,alpha,Module1,A1,78
1,S001,alpha,Module1,A2,84
2,S001,alpha,Module2,A1,79
3,S001,alpha,Module2,A2,86
4,S001,alpha,Module3,A1,81
...,...,...,...,...,...
295,S050,beta,Module1,A2,77
296,S050,beta,Module2,A1,73
297,S050,beta,Module2,A2,76
298,S050,beta,Module3,A1,75


In [59]:
scores.shape[0] >= 300

True

In [60]:
scores.head(5)

,student_id,cohort,module,assignment,score
0,S001,alpha,Module1,A1,78
1,S001,alpha,Module1,A2,84
2,S001,alpha,Module2,A1,79
3,S001,alpha,Module2,A2,86
4,S001,alpha,Module3,A1,81


In [61]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   student_id  300 non-null    object
 1   cohort      300 non-null    object
 2   module      300 non-null    object
 3   assignment  300 non-null    object
 4   score       300 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 11.8+ KB


In [62]:
scores['score'].dtypes

dtype('int64')

Task 2: Build a student roster table and merge

In [63]:
roster = pd.DataFrame({
    'student_id': [
        'S001', 'S002', 'S003', 'S004', 'S005',
        'S051', 'S052', 'S053', 'S054', 'S055',
        'S999', 'S998', 'S997', 'S996', 'S995'  # Not present in scores
    ],
    'status': [
        'active', 'active', 'inactive', 'active', 'inactive',
        'active', 'inactive', 'active', 'inactive', 'active',
        'inactive', 'active', 'inactive', 'active', 'inactive'
    ]
})
roster

,student_id,status
0,S001,active
1,S002,active
2,S003,inactive
3,S004,active
4,S005,inactive
5,S051,active
6,S052,inactive
7,S053,active
8,S054,inactive
9,S055,active


In [64]:
merged_data = pd.merge(scores, roster, on='student_id', how='left')
merged_data

,student_id,cohort,module,assignment,score,status
0,S001,alpha,Module1,A1,78,active
1,S001,alpha,Module1,A2,84,active
2,S001,alpha,Module2,A1,79,active
3,S001,alpha,Module2,A2,86,active
4,S001,alpha,Module3,A1,81,active
...,...,...,...,...,...,...
295,S050,beta,Module1,A2,77,NaN
296,S050,beta,Module2,A1,73,NaN
297,S050,beta,Module2,A2,76,NaN
298,S050,beta,Module3,A1,75,NaN


In [65]:
missing_status_count = merged_data['status'].isna().sum()
missing_status_count


np.int64(270)

In [66]:
print('Do missing statuses exist?:' , missing_status_count>0)

Do missing statuses exist?: True


Task 3: Aggregate by module and cohort 


In [67]:
avg_by_cohort= merged_data.groupby('cohort')['score'].mean().reset_index()
avg_by_module = merged_data.groupby(['cohort', 'module'])['score'].mean().reset_index()

In [68]:
avg_by_cohort

,cohort,score
0,alpha,78.715686
1,beta,79.029412
2,gamma,76.843750


In [69]:
avg_by_module

,cohort,module,score
0,alpha,Module1,77.970588
1,alpha,Module2,78.382353
2,alpha,Module3,79.794118
3,beta,Module1,78.647059
4,beta,Module2,78.411765
5,beta,Module3,80.029412
6,gamma,Module1,76.218750
7,gamma,Module2,76.468750
8,gamma,Module3,77.843750


Task 4: Reshape to a wide report

In [70]:
student_module_report = (
    merged_data.pivot_table(
        index='student_id', 
        columns='module',
        values='score',
        aggfunc='mean'
    )
)
print(student_module_report.head(5))

module      Module1  Module2  Module3
student_id                           
S001           81.0     82.5     84.5
S002           73.5     72.0     75.0
S003           87.5     88.5     89.5
S004           68.5     67.5     69.5
S005           81.5     82.5     83.5


In [71]:
merged_data['student_id'].nunique() == student_module_report.shape[0]

True

Task 5: Ranking and top performers 

In [72]:
student_avgs = scores.groupby(['student_id', 'cohort'])['score'].mean().reset_index()

top_students = student_avgs.sort_values(['cohort', 'score'], ascending=[True, False])
top_students = top_students.groupby('cohort').head(3).reset_index(drop=True)

top_students

check = top_students['cohort'].value_counts()
print(f"\nEach cohort appears 3 times: {(check == 3).all()}")



Each cohort appears 3 times: True
